In [1]:
from trade_controller import TradeController
from kraken_wsclient_py import WssClient as WsClient

import numpy as np
import pandas as pd
from datetime import datetime

from sb3_contrib import RecurrentPPO

In [2]:
class LiveTradeDeployment:
    def __init__(self, portfolio_size=3, window_size=7):
        self.bot = RecurrentPPO.load('final_bot')
        self.controller = TradeController()
        self.cash_amount = self.controller.get_account_balance()
        self.portfolio_size = portfolio_size

        self.window_size = window_size
        self.channel_mapping = {}
        self.currency_inventory = {}
        self.cash_allocation = self.cash_amount / self.portfolio_size

        self.currency_pairs = {'AVAX/USD': 'AVAXUSD',
                               'DOT/USD': 'DOTUSD',
                               'SOL/USD': 'SOLUSD'}
    
    def inventory_init(self):
        self.currency_inventory = {
            pair: 
            {
                "OHLCVT": pd.DataFrame({
                    'Time': [],
                    'Open': [], 'High': [], 'Low': [],
                    'Close': [], 'Volume': [], 'Trades': []}),
                
                "sin_time": 0, "cos_time": 0,

                "cash": self.cash_allocation, "profit": 0, "shares": 0,

                "last buy": 0, "last sell": 0, 
                "last buy id": None, "last sell id": None,

                "position open": 0, 'order min': float(self.controller.get_tradable_asset_pairs(pairs=self.currency_pairs[pair])['result'][self.currency_pairs[pair]]['ordermin'])
            } 
            for pair in self.currency_pairs}
        
        
    def data_init(self):
        for pair in self.currency_pairs:
            starter_data = self.controller.get_ohlc_data(self.currency_pairs[pair], self.controller.get_server_time() - 35 * 60, 5)['result'][self.currency_pairs[pair]][-7:]
            cleaned_data = [candle[:5] + candle[6:] for candle in starter_data]
            self.currency_inventory[pair]['OHLCVT'] = pd.concat([self.currency_inventory[pair]['OHLCVT'], pd.DataFrame(cleaned_data, columns=self.currency_inventory[pair]['OHLCVT'].columns, dtype=float)])
            self.calculate_features(self.currency_inventory[pair]['OHLCVT'])


    def calculate_rsi(self, df, period=5):
        df['delta'] = df['Close'].diff(1)
        df['gain'] = df['delta'].clip(lower=0)
        df['loss'] = df['delta'].clip(upper=0)
        avg_gain = df['gain'].ewm(com=period-1, adjust=False, min_periods = period).mean()
        avg_loss = abs(df['loss'].ewm(com=period-1, adjust=False, min_periods = period).mean())
        rs = avg_gain / avg_loss
        df['rsi'] = 100 - (100 / (1 + rs))
        df.drop(columns=['delta', 'gain', 'loss'], inplace=True)


    def calculate_macd(self, df, fast=3, slow=5, signal=2):
        fast_ema = df['Close'].ewm(span=fast, min_periods=fast, adjust=False).mean()
        slow_ema = df['Close'].ewm(span=slow, min_periods=slow, adjust=False).mean()
        df['macd'] = fast_ema - slow_ema
        df['signal_line'] = df['macd'].ewm(span=signal, min_periods=fast, adjust=False).mean()
        df['macd_h'] = df['macd'] - df['signal_line']

    
    def reformat_time_feature(self, df):
        df['Human Time'] = df['Time'].apply(lambda x: datetime.fromtimestamp(x))
        df['Human Time'] = df['Human Time'].dt.hour * 60 + df['Human Time'].dt.minute
        df['cos_time'] = np.cos(2 * np.pi * df['Human Time'] / 1440)
        df['sin_time'] = np.sin(2 * np.pi * df['Human Time'] / 1440)

    
    def calculate_pct_change_and_vol(self, df):
        df[['o_pct', 'h_pct', 'l_pct', 'c_pct', 'v_pct', 't_pct']] = df[['Open', 'High', 'Low', 'Close', 'Volume', 'Trades']].pct_change()
        df[['o_pct', 'h_pct', 'l_pct', 'c_pct', 'v_pct', 't_pct']] = df[['o_pct', 'h_pct', 'l_pct', 'c_pct', 'v_pct', 't_pct']].replace(np.inf, 100)
        df[['o_pct', 'h_pct', 'l_pct', 'c_pct', 'v_pct', 't_pct']] = df[['o_pct', 'h_pct', 'l_pct', 'c_pct', 'v_pct', 't_pct']].replace(-np.inf, -100)
        df['vol'] = df['High'] / df['Low']
        df['vol'] = df['vol'].replace(np.inf, 100)
        df['vol'] = df['vol'].replace(-np.inf, -100)

    
    def calculate_features(self, df):
        self.calculate_macd(df)
        self.calculate_rsi(df)
        self.calculate_pct_change_and_vol(df)
        self.reformat_time_feature(df)

    
    def update_frame(self, pair):
        self.currency_inventory[pair]['OHLCVT'] = self.currency_inventory[pair]['OHLCVT'].tail(self.window_size)
        self.calculate_features(self.currency_inventory[pair]['OHLCVT'])

        
    def build_state(self, pair):
        market_state = self.currency_inventory[pair]['OHLCVT'][['Open', 'High', 'Low', 'Close', 'Volume', 'Trades',
                    'o_pct', 'h_pct', 'l_pct', 'c_pct', 'v_pct', 't_pct',
                    'vol', 'rsi', 'macd', 'signal_line', 'macd_h', 'cos_time', 'sin_time']].iloc[-1].to_numpy()
        
        
        inventory_state = np.array([self.currency_inventory[pair]['cash'], self.currency_inventory[pair]['profit'],
                                    self.currency_inventory[pair]['shares'], self.currency_inventory[pair]['last buy'],
                                    self.currency_inventory[pair]['last sell'], self.currency_inventory[pair]['position open']])
        
        
        full_state = np.append(market_state, inventory_state)
        return full_state
    
    

    def execute_trade(self, pair):
        state = self.build_state(pair)
        action, _ = self.bot.predict(state, deterministic=True)
        volume = self.currency_inventory[pair]['cash'] / state[3]

        volume =  self.currency_inventory[pair]['order min'] if volume < self.currency_inventory[pair]['order min'] else volume
         
        if action == 0 and not self.currency_inventory[pair]['position open']:
            response = self.controller.add_order(pair=self.currency_pairs[pair], type='buy', volume=volume)

            if response['error']:
                raise Exception(f"Error in Buy Execution: {response['error']}")

            transaction_id = response['result']['txid'][0]
            transaction_info = self.controller.query_orders(txid=transaction_id)['result'][transaction_id]

            self.currency_inventory[pair]['last buy id'] = transaction_id

            self.currency_inventory[pair]['shares'] += float(transaction_info['vol_exec'])
            self.currency_inventory[pair]['last buy'] = float(transaction_info['price']) * self.currency_inventory[pair]['shares']
            self.currency_inventory[pair]['cash'] -= self.currency_inventory[pair]['last buy']
            self.currency_inventory[pair]['position open'] = 1

            print(f"Bought {pair} at {self.currency_inventory[pair]['last buy']}")
            

        elif action == 1 and self.currency_inventory[pair]['position open']:
            buy_id = self.currency_inventory[pair]['last buy id']
            buy_info = self.controller.query_orders(txid=buy_id)['result'][buy_id]

            volume = buy_info['vol']
            response = self.controller.add_order(pair=self.currency_pairs[pair], type='sell', volume=volume)

            if response['error']:
                raise Exception(f"Error in Sell Execution: {response['error']}")
            
            transaction_id = response['result']['txid'][0]
            transaction_info = self.controller.query_orders(txid=transaction_id)['result'][transaction_id]
            self.currency_inventory[pair]['last sell id'] = transaction_id

            self.currency_inventory[pair]['last sell'] = float(transaction_info['price']) * float(transaction_info['vol_exec'])
            self.currency_inventory[pair]['shares'] -= float(transaction_info['vol_exec'])
            self.currency_inventory[pair]['cash'] += self.currency_inventory[pair]['last sell']
            self.currency_inventory[pair]['profit'] += self.currency_inventory[pair]['last sell'] - self.currency_inventory[pair]['last buy']
            self.currency_inventory[pair]['position open'] = 0
            
            print(f"Sold {pair} and made ${self.currency_inventory[pair]['profit']} profit")

        else:
            print(f'Holding position for pair {pair}')
        

    def websocket_handler(self, message):
        if isinstance(message, dict) and 'channelID' in message.keys() and 'pair' in message.keys():
            self.channel_mapping[message['channelID']] = str(message['pair'])

        elif isinstance(message, list):
            currency_df = self.currency_inventory[self.channel_mapping[message[0]]]['OHLCVT']
            candle_update = pd.DataFrame([message[1][1:6] + message[1][7:] + [0] * 14], columns=currency_df.columns, dtype=float)
            previous_time = currency_df['Time'].iloc[-1]
            current_time = int(float(message[1][1]))
            pair = self.channel_mapping[message[0]]

            if current_time - previous_time > 60:
                self.currency_inventory[pair]['OHLCVT'] = pd.concat([currency_df, candle_update], ignore_index=True)
                self.update_frame(pair)
                self.execute_trade(pair)


    def websocket_start(self, name=None):
        my_client = WsClient()
        my_client.start()

        my_client.subscribe_public(
            subscription={
                'name': name,
                'interval': 5
            },
            pair=[pair for pair in self.currency_pairs],
            callback=self.websocket_handler
        )

    def deploy_to_live(self):
        self.inventory_init()
        self.data_init()
        self.websocket_start('ohlc')

In [3]:
live = LiveTradeDeployment()

In [4]:
live.deploy_to_live()

Bought AVAX/USD at 13.605
Holding position for pair DOT/USD
Holding position for pair SOL/USD
Holding position for pair DOT/USD
Holding position for pair SOL/USD
Holding position for pair AVAX/USD
Holding position for pair DOT/USD
Sold AVAX/USD and made $-0.025000000000000355 profit
Holding position for pair SOL/USD
Holding position for pair AVAX/USD
Holding position for pair SOL/USD
Holding position for pair DOT/USD
Holding position for pair DOT/USD
Holding position for pair AVAX/USD
Holding position for pair SOL/USD
Holding position for pair SOL/USD
Holding position for pair AVAX/USD
Holding position for pair DOT/USD
Holding position for pair SOL/USD
Holding position for pair DOT/USD
Holding position for pair AVAX/USD
Holding position for pair SOL/USD
Holding position for pair AVAX/USD
Bought DOT/USD at 9.448866652085
Holding position for pair DOT/USD
Holding position for pair SOL/USD
Holding position for pair AVAX/USD
Holding position for pair SOL/USD
Sold DOT/USD and made $-0.00482